<a href="https://colab.research.google.com/github/mamathasara/mamathasara-business-faq-rag-bot/blob/main/business_faq_rag_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 STEP 1: Install Required Libraries

In [1]:
!pip install -U pinecone-client sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

❓ Why I Shifted from OpenAI Embeddings to Hugging Face Sentence Transformers
Initially, I planned to use OpenAI’s text-embedding-3-small model for generating embeddings in my Retrieval-Augmented Generation (RAG) system. However, during implementation, I encountered the following issue:

❌ RateLimitError: “You exceeded your current quota...”

This error occurred because I had exhausted my free quota on the OpenAI API and had not set up a paid billing method. Since the assignment required a working RAG model — and I wanted to continue without incurring costs — I decided to shift to a free and reliable alternative.

In [3]:
!pip uninstall -y pinecone-client
!pip install pinecone


Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 24.3 MB/s eta 0:00:00


✅ STEP 2: Import Libraries and Set API Keys

✅ STEP 3: Create or Connect to Pinecone Index


In [5]:
from pinecone import Pinecone, ServerlessSpec

# Create Pinecone client object
pc = Pinecone(api_key="pcsk_22VKRh_NVKxv12BGVpzsbqBKCnma5Jok5M48h4P8QLGquumK1RzJefKbU8uGYtFwXinhvb")

# Create index if it doesn’t exist
if "rag-qa-bot" not in pc.list_indexes().names():
    pc.create_index(
        name="rag-qa-bot",
        dimension=384,  # For sentence-transformers model
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the index
index = pc.Index("rag-qa-bot")


✅ STEP 4: Prepare Business FAQs

In [6]:
docs = [
    "Our business hours are 9 AM to 5 PM, Monday through Friday.",
    "You can contact our customer support by emailing support@ourcompany.com.",
    "We offer a 30-day return policy for all products purchased through our website.",
    "Shipping typically takes 5–7 business days for standard delivery.",
    "Our company specializes in custom software solutions for e-commerce businesses.",
    "You can reset your account password using the 'Forgot Password' link on the login page.",
    "We provide onboarding sessions for new clients every Monday at 11 AM via Zoom.",
    "Invoices are generated automatically at the end of each billing cycle and can be downloaded from your dashboard.",
    "Employees are eligible for health insurance benefits after 3 months of full-time employment.",
    "To cancel your subscription, go to the billing settings in your account dashboard and click 'Cancel Plan'.",
    "We provide 24/7 live chat support for premium customers.",
    "Our data privacy policy complies with GDPR and CCPA regulations.",
    "Bulk orders above 100 units are eligible for custom discounts. Contact sales@ourcompany.com for more info.",
    "You can track your order status using the tracking link sent to your registered email.",
    "Our mobile app is available on both iOS and Android platforms.",
    "We host monthly webinars on topics like business automation and AI in CRM systems.",
    "All employees are required to complete annual security awareness training.",
    "Technical documentation for our APIs is available on our developer portal.",
    "Refunds are processed within 5–10 business days after receiving returned items.",
    "We partner with Stripe for secure online payments and subscription management."
]


✅ STEP 5: Generate Embeddings & Upload to Pinecone (Free + Fast)

In [9]:
if "rag-qa-bot" in pc.list_indexes().names():
    pc.delete_index("rag-qa-bot")


I am deleting the existing Pinecone index because it was created using OpenAI embeddings with a dimension of 1536. Since I am now switching to a Hugging Face model, which uses a different embedding dimension, I need to create a new index with the appropriate configuration.

In [10]:
from pinecone import ServerlessSpec

pc.create_index(
    name="rag-qa-bot",
    dimension=384,  # ✅ Matches Hugging Face embedding size
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


{
    "name": "rag-qa-bot",
    "metric": "cosine",
    "host": "rag-qa-bot-28mdvhu.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [11]:
index = pc.Index("rag-qa-bot")


In [12]:
# Load Hugging Face model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate 384-dim embeddings
vectors = model.encode(docs).tolist()

# Upload to Pinecone using new SDK format
for i, (text, vector) in enumerate(zip(docs, vectors)):
    index.upsert(
        vectors=[
            {
                "id": str(i),
                "values": vector,
                "metadata": {"text": text}
            }
        ]
    )


✅ STEP 6: Answer Generation using Flan-T5 (Free Hugging Face Model)


In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch


In [15]:
# Load FLAN-T5 (small model for speed on Colab)
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
def generate_answer_flant5(question, context):
    # Create the prompt with context
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    # Generate answer
    outputs = hf_model.generate(**inputs, max_new_tokens=100)

    # Decode the output
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer


In [17]:
def retrieve_relevant_docs(question, top_k=3):
    question_vector = model.encode(question).tolist()
    results = index.query(vector=question_vector, top_k=top_k, include_metadata=True)
    return [match['metadata']['text'] for match in results['matches']]


In [18]:
def rag_pipeline_free(question):
    # Retrieve similar docs
    retrieved_docs = retrieve_relevant_docs(question)

    # Combine as single context block
    context = "\n".join(retrieved_docs)

    # Generate answer using Flan-T5
    answer = generate_answer_flant5(question, context)

    return answer


In [25]:
question = "How can I reset my password?"
answer = rag_pipeline_free(question)

print("Q:", question)
print("A:", answer)


Q: How can I reset my password?
A: Forgot Password


In [26]:
!pip install gradio

import gradio as gr

def rag_ui(question):
    return rag_pipeline_free(question)

# Build the Gradio interface
interface = gr.Interface(
    fn=rag_ui,
    inputs=gr.Textbox(lines=2, placeholder="Ask a business-related question..."),
    outputs="text",
    title="📚 Business FAQ RAG Bot (Free)",
    description="Ask your business-related questions — powered by Hugging Face + Pinecone."
)

# Launch and get link
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f7b0f0671192d40755.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
